In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
data=pd.read_csv('GEFCom2014N.csv').dropna()
data['load']=data['load']/10000
data['T']=data['T']/100
# Convert timestamp to datetime format
data['Datetime'] = data['Datetime'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

# Set the datetime as index
data.set_index('Datetime', inplace=True)

# Resample the data to hourly frequency
data = data.resample('H').mean()

# Create lag features
for i in range(1, 25):
    data['lag_{}'.format(i)] = data['load'].shift(i)

# Create rolling mean and standard deviation features
data['rolling_mean'] = data['load'].rolling(window=24).mean()
data['rolling_std'] = data['load'].rolling(window=24).std()

# Create weekday and hour features
data['weekday'] = data.index.weekday
data['hour'] = data.index.hour

# Remove missing values
data.dropna(inplace=True)

# Split the data into train and test sets
train_size = int(len(data) * 0.8)
train_data = data[:train_size]
test_data = data[train_size:]

In [4]:
#Attention
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LSTM, Conv1D, MaxPooling1D, Flatten, Dropout, Attention
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Prepare the train and test data
X_train, y_train = train_data.drop('load', axis=1), train_data['load']
X_test, y_test = test_data.drop('load', axis=1), test_data['load']

# Reshape the data 
X_train = X_train.values.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.values.reshape(X_test.shape[0], X_test.shape[1], 1)

# Define the input shape
input_shape = (X_train.shape[1], 1)

# Define the model
inputs = Input(shape=input_shape)
conv1 = Conv1D(filters=64, kernel_size=3, activation='relu')(inputs)
pool1 = MaxPooling1D(pool_size=2)(conv1)
lstm1 = LSTM(64, return_sequences=True)(pool1)
attention = Attention()([lstm1, lstm1])
flatten = Flatten()(attention)
dropout = Dropout(0.2)(flatten)
outputs = Dense(1)(dropout)
model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(loss='mse', optimizer=Adam())

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test))

# Evaluate the model
from sklearn.metrics import mean_absolute_error, mean_squared_error
mae = history.history['val_loss'][-1]
mse = mean_squared_error(y_test, model.predict(X_test))
rmse = np.sqrt(mse)

ypred = model.predict(X_test)
y_test=test_data['load']
y_attention=ypred


Epoch 1/50
986/986 [==============================] - 14s 12ms/step - loss: 0.0024 - val_loss: 5.5712e-04
Epoch 2/50
986/986 [==============================] - 12s 12ms/step - loss: 3.3927e-04 - val_loss: 3.4815e-04
Epoch 3/50
986/986 [==============================] - 12s 12ms/step - loss: 2.3224e-04 - val_loss: 1.0647e-04
Epoch 4/50
986/986 [==============================] - 12s 12ms/step - loss: 2.0093e-04 - val_loss: 1.8305e-04
Epoch 5/50
986/986 [==============================] - 12s 12ms/step - loss: 1.6063e-04 - val_loss: 1.3931e-04
Epoch 6/50
986/986 [==============================] - 12s 12ms/step - loss: 1.3152e-04 - val_loss: 6.2703e-05
Epoch 7/50
986/986 [==============================] - 12s 12ms/step - loss: 1.1880e-04 - val_loss: 6.6492e-05
Epoch 8/50
986/986 [==============================] - 12s 12ms/step - loss: 1.0752e-04 - val_loss: 8.0896e-05
Epoch 9/50
986/986 [==============================] - 12s 12ms/step - loss: 8.9053e-05 - val_loss: 6.0550e-05
Epoch 10/50
98

In [5]:
#For MAE
mae = np.mean(np.abs(ypred.ravel() - y_test.ravel()))
print('MAE = {:.2f} (%)'.format(mae))

#For MAPE
mape = np.mean(np.abs((y_test.ravel() - ypred.ravel()) / y_test.ravel())) * 100
print('MAPE = {:.2f} (%)'.format(mape))


# For RMSE
from sklearn.metrics import mean_squared_error 
mse=mean_squared_error(y_test,ypred) 
rsme=np.sqrt(mse) 
print('RSME = {:.2f} (%)'.format(rsme))

MAE = 0.00 (%)
MAPE = 1.26 (%)
RSME = 0.01 (%)
